In [3]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable
Using legacy 'setup.py install' for pyperclip, since package 'wheel' is not installed.
    Running setup.py install for pyperclip: started
    Running setup.py install for pyperclip: finished with status 'done'


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


In [6]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


In [7]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv('/02_Work/workspace/Datasets/Admission_Prediction.csv')

In [10]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [23]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [25]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [31]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [26]:
x = df.drop(columns=['Serial No.', 'Chance of Admit'])
y = df['Chance of Admit']

In [27]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [28]:
from sklearn.metrics import mean_squared_error
def objective(trail, data = x, target= y):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=.10, random_state=30)
    params={
        'tree_method' :'gpu_hist',
        'lamda': trail.suggest_loguniform('lambda', 1e-4, 10.0),
        'alpha': trail.suggest_loguniform('alpha', 1e-4, 10.0),
        'colsample_bytree':trail.suggest_categorical('colsample_bytree', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample':trail.suggest_categorical('subsample', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate':trail.suggest_categorical('learning_rate', [.00001, .0003, .0008, .02, .01, 1, 10, 20]),
        'n_estimator':30000,
        'max_depth':trail.suggest_categorical('max_depth', [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
        'random_state':trail.suggest_categorical('random_state', [10, 20, 30, 2000, 3454, 243123]),
        'min_child_weight':trail.suggest_int('min_child_weight', 1, 200),
        'verbosity':3
    }

    xgb_reg_model = xgb.XGBRegressor(**params)
    xgb_reg_model.fit(train_x, train_y, eval_set=[(test_x, test_y)], verbose=True)
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y, pred_xgb)
    return rmse
    

In [29]:
find_param = optuna.create_study(direction='minimize')

[I 2021-10-31 06:37:17,727] A new study created in memory with name: no-name-759d1ffd-649b-4429-a93e-f98aa4504d52


In [30]:
find_param.optimize(objective, n_trials=10)
find_param.best_trial.params

[W 2021-10-31 06:37:24,597] Trial 0 failed because of the following error: XGBoostError('[06:37:24] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:588: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.')
Traceback (most recent call last):
  File "C:\Users\adhik\AppData\Roaming\Python\Python39\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-28-c9abdf8acabd>", line 19, in objective
    xgb_reg_model.fit(train_x, train_y, eval_set=[(test_x, test_y)], verbose=True)
  File "C:\Users\adhik\AppData\Roaming\Python\Python39\site-packages\xgboost\core.py", line 506, in inner_f
    return f(**kwargs)
  File "C:\Users\adhik\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py", line 789, in fit
    self._Booster = train(
  File "C:\Users\adhik\AppData\Roaming\Python\Python39\site-packages\xgboost\training.py", line 188, in train
    bst =

[06:37:24] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/tree/updater_gpu_hist.cu:872: [GPU Hist]: Configure
[06:37:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[06:37:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "lamda", "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBoostError: [06:37:24] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:588: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.